In [7]:
import nltk
import pandas as pd
import numpy as np


train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sub = pd.read_csv('sample_submission.csv')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/jake/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jake/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [54]:
from io import StringIO

col = ['author', 'text']
train = train[pd.notnull(train['text'])]


train['author_id'] = train['author'].factorize()[0]
author_id_df = train[['author', 'author_id']].drop_duplicates().sort_values('author_id')
author_to_id = dict(author_id_df.values)
id_to_category = dict(author_id_df[['author_id', 'author']].values)

author_id = train['author_id']
proper_noun_list  = []
proper_nouns = []
for sentence in train['text']:
    tokens = nltk.word_tokenize(sentence)
    pos = nltk.pos_tag(tokens)
    proper_noun = []
    for tag in pos:
        if (tag[1] == "NNP"):
            proper_noun_list.append(tag[0])
            proper_noun.append(tag[0])
    proper_nouns.append(proper_noun)
    
    
proper_noun_list = list(dict.fromkeys(proper_noun_list))
         
print(proper_noun_list)

['Windsor', 'Terrace', 'Superintendent', 'Epicurus', 'Greek', 'Orion', 'A', 'Herbert', 'West', 'Wheaton', 'Street', 'Draconian', 'Pestilence', 'Speculation', 'California', 'Von', 'Kempelen', 'Sarnath', 'Ib', 'Mnar', 'Paradise', 'Arnheim', 'Their', 'Sheehan', 'Old', 'Bugs', 'River', 'Innsmouth', 'D', 'Wilbur', 'Monday', 'Merrival', 'Underduk', 'Dr.', 'Johnson', 'Man', 'Aspect', 'Ellison', 'Nature', 'Mein', 'Gott', 'No', 'Fire', 'O', 'Raymond', 'Magazine', 'Constantinople', 'Greeks', 'G', 'Ahead', 'William', 'Latin', 'Cornelius', 'Agrippa', 'Albertus', 'Magnus', 'Paracelsus', 'Idris', 'Adrian', 'Legrand', 'Geneva', 'Mr.', 'Theodore', 'Sinivate', 'Gawd', 'Oh', 'Comte', 'Charles', 'Le', 'Sorcier', 'Perdita', 'Scotland', 'God', 'Ashimah', 'Main', 'Protectorate', 'Earl', 'Nith', 'Ingolstadt', 'Kidd', 'Look', 'Godhead', 'Galpin', 'New', 'York', 'University', 'English', 'Kilderry', 'Lofoden', 'Moskoe', 'Cumberland', 'Greece', 'Pierre', 'Moreau', 'Madame', "L'Espanaye", 'Hebrews', 'Heaven', 'Am

In [55]:
count = 0
for pn in proper_nouns:
    if(len(pn) != 0):
        count += 1

print(count/len(proper_nouns))
print(len(proper_noun_list))

0.36028397773124266
4084


In [56]:
def find_links(arr_nouns):
    nouns = []
    for noun1 in arr_nouns:
        for noun2 in arr_nouns:
            if(noun1 != noun2):
                nouns.append([noun1,noun2])
    return nouns


                
    
proper_nouns_linked = []
for pnn in proper_nouns:
    proper_nouns_linked.append(find_links(pnn))

    

In [75]:
eap_count = []
hpl_count = []
mws_count = []
total_count = []
links = []

for z in range(len(proper_noun_list)):
    noun = proper_noun_list[z]
    eap_count.append(0)
    hpl_count.append(0)
    mws_count.append(0)
    total_count.append(0)
    for i in range(len(author_id)):
        for noun2 in proper_nouns[i]:
            if(noun == noun2):
                if(author_id[i] == 0):
                    eap_count[z] += 1
                if(author_id[i] == 1):
                    hpl_count[z] += 1
                if(author_id[i] == 2):
                    mws_count[z] += 1
                total_count[z] += 1
            
    

       

In [76]:
from collections import Counter

links = []
for z in range(len(proper_noun_list)):
    noun = proper_noun_list[z]
    cnt = Counter()
    for all_noun_links in proper_nouns_linked:
        for noun_links in all_noun_links:
            if(noun == noun_links[0]):
                cnt[noun_links[1]] += 1
    links.append(cnt)

    
          

In [79]:
df = pd.DataFrame(zip(proper_noun_list,total_count,eap_count,hpl_count,mws_count,links), columns = ['Noun','TotalCount','EAP', 'HPL', "MWS", "Links"]) 




In [81]:
proper_nouns
    

[[],
 [],
 [],
 ['Windsor', 'Terrace'],
 ['Superintendent'],
 [],
 [],
 [],
 ['Epicurus', 'Greek', 'Orion'],
 [],
 ['A'],
 [],
 ['Herbert', 'West'],
 ['Wheaton', 'Street'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['Draconian'],
 [],
 ['Pestilence'],
 ['Speculation', 'California', 'Von', 'Kempelen'],
 [],
 [],
 ['Sarnath', 'Ib', 'Mnar'],
 ['Paradise', 'Arnheim'],
 [],
 [],
 ['Their'],
 ['Sheehan', 'Old', 'Bugs'],
 [],
 ['River', 'Street', 'Innsmouth'],
 [],
 [],
 ['D'],
 ['Wilbur'],
 [],
 ['Monday'],
 ['Merrival', 'Windsor'],
 ['Underduk'],
 [],
 [],
 [],
 ['Dr.', 'Johnson', 'Man', 'Aspect'],
 [],
 ['Ellison'],
 [],
 [],
 [],
 [],
 ['Nature'],
 [],
 ['Mein', 'Gott', 'No'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['Fire', 'O', 'Raymond'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['Raymond'],
 ['Magazine'],
 [],
 [],
 ['Constantinople', 'Greeks'],
 [],
 [],
 [],
 [],
 [],
 ['G'],
 ['Ahead'],
 [],
 ['William'],
 ['Latin'],
 ['Cornelius', 'Agrippa', 'Albertus', 'Magnus', 'Paracelsus'],
 [

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

2344
2345
2346
2347
2348
2349
2350
2351
2352
2353
2354
2355
2356
2357
2358
2359
2360
2361
2362
2363
2364
2365
2366
2367
2368
2369
2370
2371
2372
2373
2374
2375
2376
2377
2378
2379
2380
2381
2382
2383
2384
2385
2386
2387
2388
2389
2390
2391
2392
2393
2394
2395
2396
2397
2398
2399
2400
2401
2402
2403
2404
2405
2406
2407
2408
2409
2410
2411
2412
2413
2414
2415
2416
2417
2418
2419
2420
2421
2422
2423
2424
2425
2426
2427
2428
2429
2430
2431
2432
2433
2434
2435
2436
2437
2438
2439
2440
2441
2442
2443
2444
2445
2446
2447
2448
2449
2450
2451
2452
2453
2454
2455
2456
2457
2458
2459
2460
2461
2462
2463
2464
2465
2466
2467
2468
2469
2470
2471
2472
2473
2474
2475
2476
2477
2478
2479
2480
2481
2482
2483
2484
2485
2486
2487
2488
2489
2490
2491
2492
2493
2494
2495
2496
2497
2498
2499
2500
2501
2502
2503
2504
2505
2506
2507
2508
2509
2510
2511
2512
2513
2514
2515
2516
2517
2518
2519
2520
2521
2522
2523
2524
2525
2526
2527
2528
2529
2530
2531
2532
2533
2534
2535
2536
2537
2538
2539
2540
2541
2542
2543


['Windsor',
 'Terrace',
 'Superintendent',
 'Epicurus',
 'Greek',
 'Orion',
 'A',
 'Herbert',
 'West',
 'Wheaton',
 'Street',
 'Draconian',
 'Pestilence',
 'Speculation',
 'California',
 'Von',
 'Kempelen',
 'Sarnath',
 'Ib',
 'Mnar',
 'Paradise',
 'Arnheim',
 'Their',
 'Sheehan',
 'Old',
 'Bugs',
 'River',
 'Innsmouth',
 'D',
 'Wilbur',
 'Monday',
 'Merrival',
 'Underduk',
 'Dr.',
 'Johnson',
 'Man',
 'Aspect',
 'Ellison',
 'Nature',
 'Mein',
 'Gott',
 'No',
 'Fire',
 'O',
 'Raymond',
 'Magazine',
 'Constantinople',
 'Greeks',
 'G',
 'Ahead',
 'William',
 'Latin',
 'Cornelius',
 'Agrippa',
 'Albertus',
 'Magnus',
 'Paracelsus',
 'Idris',
 'Adrian',
 'Legrand',
 'Geneva',
 'Mr.',
 'Theodore',
 'Sinivate',
 'Gawd',
 'Oh',
 'Comte',
 'Charles',
 'Le',
 'Sorcier',
 'Perdita',
 'Scotland',
 'God',
 'Ashimah',
 'Main',
 'Protectorate',
 'Earl',
 'Nith',
 'Ingolstadt',
 'Kidd',
 'Look',
 'Godhead',
 'Galpin',
 'New',
 'York',
 'University',
 'English',
 'Kilderry',
 'Lofoden',
 'Moskoe',
 'C